# 一些实例

本文主要记录一些平常自己写的，在实际中使用的代码。

## 计算流域平均气象时间序列数据

这里以Daymet 2天的网格数据，CAMELS 多个流域为例，计算这些流域这两天每日的forcing数据流域平均值。

因为CAMELS文件比较大，所以这里没有传到github上，需要手动从[这里](https://ral.ucar.edu/sites/default/files/public/product-tool/camels-catchment-attributes-and-meteorology-for-large-sample-studies-dataset-downloads/basin_set_full_res.zip)下载 CAMELS 的shpfile ，然后在本文件夹下创建一个large_files文件夹，并将下好的CAMELS shpfile放到其中。

也可以选择上传到GEE asset上，然后直接调用。

In [1]:
import ee
import geemap
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
# Add Earth Engine dataset
daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4")

In [3]:
# 本地文件
# camels_shp = 'large_files/HCDN_nhru_final_671.shp'
# camels = geemap.shp_to_ee(camels_shp)
# 远程asset上
camels = ee.FeatureCollection("users/wenyu_ouyang/HCDN_nhru_final_671")
camels

In [4]:
#maybe better to use Number to replace js number
year = ee.Number(2000)
month = ee.Number(1)
day = ee.Number(1)
start_date = ee.Date.fromYMD(year, month, day)
end_date = start_date.advance(2, 'day')
end_date

In [5]:
days_num = end_date.difference(start_date, 'day')
# count day from zero, and ee.List.sequence is a closed interval
days = ee.List.sequence(ee.Number(0),days_num.add(-1))
# get Imagecollection and filter, choose two days for test
daymet_days = daymet.filter(ee.Filter.date(start_date, end_date))

# show maximumTemperature, just for test
maximumTemperature = daymet_days.select('tmax')
maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}
Map.setCenter(-110.21, 35.1, 4)
Map.addLayer(maximumTemperature, maximumTemperatureVis, 'Maximum Temperature')

In [8]:
def nestedMappedReducer(featCol, imgCol):
    def mapReducerOverImgCol(feat):
        def imgReducer(img):
            vals = img.reduceRegion(
                reducer = ee.Reducer.mean(),
                geometry = feat.geometry(),
                scale = 1000
            )
            return ee.Feature(None, vals).set({
                "system:time_start": img.get("system:time_start"),
                "hru_id" : feat.get("hru_id")
            })
        return imgCol.map(imgReducer);
    return featCol.map(mapReducerOverImgCol).flatten()

执行函数并导出到 google drive，这样本地可以关闭，远程也在运行了，很适合较长时间的计算。

In [9]:
daymet_regions = nestedMappedReducer(camels, daymet_days)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=daymet_regions, description="daymet_camels_mean_20000101-02new", folder="export", file_format="csv", 
    selectors=["hru_id","system:time_start","dayl","prcp","srad","swe","tmax","tmin","vp"]
)

Exporting daymet_camels_mean_20000101-02new...


来自GEE的建议，使用reduceRegions可能比reduceRegion更快一些，但是这里不方便设置每个feature的time_start和hru_id，应该还是需要对featurecollection加一个map函数，所以暂时就不尝试了。

```Javascript
results = daymet_days.map(function(img) {
    return img.reduceRegions({
        ...
    });
});
```

上面给出的是JS代码，关于map和reduce，尤其是嵌套的js代码转python代码，可以参考这里：https://gis.stackexchange.com/questions/365121/how-to-nest-mapped-functions-with-the-earth-engine-python-api